<a href="https://colab.research.google.com/github/forowho/Dendritic-and-non_Dendritic-Micrograph-Classification/blob/main/Dendritic_Micrograph_classification_model_comparison_No_Aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import skimage.io
import os
%matplotlib inline
import shutil
from shutil import copyfile
import random
import time
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Import dataset from google drive

In [ ]:
# Download dataset
!wget https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/nyb6mycvfd-1.zip

--2024-10-18 21:42:23--  https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/nyb6mycvfd-1.zip
Resolving prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)... 3.5.66.31, 52.218.62.72, 52.218.46.106, ...
Connecting to prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)|3.5.66.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 111332105 (106M) [application/octet-stream]
Saving to: ‘nyb6mycvfd-1.zip.1’

nyb6mycvfd-1.zip.1  100%[===================>] 106.17M  24.8MB/s    in 4.5s    

2024-10-18 21:42:29 (23.7 MB/s) - ‘nyb6mycvfd-1.zip.1’ saved [111332105/111332105]



In [ ]:
#unzip the dataset
!unzip -q /content/nyb6mycvfd-1.zip

replace Dendritic/10.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace Dendritic/2.1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace Dendritic/2.1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
# setting path to the main directory
main_dir = '/content/Dendrite micrograph'

In [ ]:
# set path to dendritic micrograph images
dendritic_dir = '/content/Dendritic'
# set path to non dendritic micrograph images
non_dendritic_dir = '/content/Non-Dendritic'

#Print the total number of images in each directory
print("The total number of dendritic images are", len(os.listdir(dendritic_dir)))
print("The total number of non dendritic images are", len(os.listdir(non_dendritic_dir)))

The total number of dendritic images are 133
The total number of non dendritic images are 443


In [ ]:
# Paths to the dataset
dendritic_folder = "/content/Dendritic"
non_dendritic_folder = "/content/Non-Dendritic"

#Step 1: Import necessary libraries

In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier as KNeighborClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

#Step 2: Define a function to extract HOG features
HOG is a common feature extraction technique, but you can modify this to include other features like color histograms, edge detection, etc.

In [ ]:
def extract_hog_features(image):
    # Convert the image to grayscale for HOG
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Extract HOG features
    features, hog_image = hog(gray_image, orientations=9, pixels_per_cell=(8, 8),
                              cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')
    return features


#Step 3: Load the dataset and extract features

* Image Loading and Processing: The cv2.imread() function loads each image from the two subfolders (Dendritic and Non-Dendritic), and HOG features are extracted using the extract_hog_features function.

* Labeling: Images from the Dendritic folder are labeled as 1 and from the Non-Dendritic folder as 0.

* SVC Model: After extracting features, the data is split into training and test sets, and the SVC is trained on the training data.

* Evaluation: The accuracy and classification report of the model are printed.

In [ ]:
import cv2
import os
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split

def extract_hog_features(image):
    # Convert the image to grayscale for HOG
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Resize the image to a fixed size for consistent HOG feature size
    resized_image = cv2.resize(gray_image, (64, 64)) # Resize to 64x64
    # Extract HOG features
    features, hog_image = hog(resized_image, orientations=9, pixels_per_cell=(8, 8),
                            cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')
    return features

def load_dataset_and_extract_features():
    data = []
    labels = []
    # Process dendritic images
    for img_name in os.listdir(dendritic_folder):
        img_path = os.path.join(dendritic_folder, img_name)
        image = cv2.imread(img_path)
        if image is not None:
            features = extract_hog_features(image)
            data.append(features)
            labels.append(1)  # 1 for Dendritic
    # Process non-dendritic images
    for img_name in os.listdir(non_dendritic_folder):
        img_path = os.path.join(non_dendritic_folder, img_name)
        image = cv2.imread(img_path)
        if image is not None:
            features = extract_hog_features(image)
            data.append(features)
            labels.append(0)  # 0 for Non-Dendritic
    return np.array(data), np.array(labels)

#Step 4: Train Test Split
We use stratify to ensure data from both the non-dendritic folder and dendritic folder use for training are proportional

In [ ]:
X, Y = load_dataset_and_extract_features()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify = Y, random_state=42)


In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(510, 1764) (408, 1764) (102, 1764)


#Step 5: Comparing the performance of models
* LogisticRegression()
* SVC(kernel='linear')
* KNeighborClassifier()
* RandomForestClassifier()

In [ ]:
models = [LogisticRegression(max_iter=1000), SVC(kernel='linear'), KNeighborClassifier(), RandomForestClassifier()]

def compare_models_train_test():
  for model in models:
    # Train the model
    model.fit(X_train, y_train)

    # Evaluate the model on training data
    train_predictions = model.predict(X_train)
    train_accuracy = accuracy_score(y_train, train_predictions)
    train_accuracy = train_accuracy*100
    train_accuracy = round(train_accuracy, 2)
    print(f"Training Accuracy for {model}: {train_accuracy}%")

    # Evaluate the model on validation data
    val_predictions = model.predict(X_test)
    val_accuracy = accuracy_score(y_test, val_predictions)
    val_accuracy = val_accuracy*100
    val_accuracy = round(val_accuracy, 2)
    print(f"Validation Accuracy for {model}: {val_accuracy}%")

compare_models_train_test()

Training Accuracy for LogisticRegression(max_iter=1000): 92.89%
Validation Accuracy for LogisticRegression(max_iter=1000): 70.59%
Training Accuracy for SVC(kernel='linear'): 95.34%
Validation Accuracy for SVC(kernel='linear'): 66.67%
Training Accuracy for KNeighborsClassifier(): 81.13%
Validation Accuracy for KNeighborsClassifier(): 64.71%
Training Accuracy for RandomForestClassifier(): 95.83%
Validation Accuracy for RandomForestClassifier(): 75.49%


# Cross Validation

In [ ]:
cv_score_lr = cross_val_score(LogisticRegression(max_iter=1000), X, Y, cv=5)

In [ ]:
print(cv_score_lr)

[0.67647059 0.66666667 0.67647059 0.7254902  0.73529412]


In [ ]:
mean_accuracy_lr = sum(cv_score_lr)/len(cv_score_lr)
mean_accuracy_lr = mean_accuracy_lr*100
mean_accuracy_lr = round(mean_accuracy_lr, 2)
print(mean_accuracy_lr)

69.61


In [ ]:
models = [LogisticRegression(), SVC(kernel='linear'), KNeighborClassifier(), RandomForestClassifier()]

def compare_models_train_test():
  for model in models:
    cv_score = cross_val_score(model, X, Y, cv=5)
    #evaluate the model
    mean_accuracy = sum(cv_score)/len(cv_score)
    mean_accuracy = mean_accuracy*100
    mean_accuracy = round(mean_accuracy, 2)
    print('Accuracy score of the ', model, ': ', mean_accuracy, '%')

compare_models_train_test()

Accuracy score of the  LogisticRegression() :  69.61 %
Accuracy score of the  SVC(kernel='linear') :  60.98 %
Accuracy score of the  KNeighborsClassifier() :  71.57 %
Accuracy score of the  RandomForestClassifier() :  73.33 %
